## Processing Summer irrigated area results

This script will produce timeseries plots of annual summer irrigated area, and annual newly developed irrigated area.

`Each sub-section has its own user inputs`

In [ ]:
import numpy as np
import xarray as xr
import geopandas as gpd
import pandas as pd
from osgeo import gdal, ogr
import os
import warnings
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pathlib import Path
from shapely.geometry import Polygon, MultiPolygon
from multiprocessing import Pool
import sys

### Annual summer irrigated area plot

In [ ]:
#------------
# User inputs
#------------

#path to the annual summer irrigated area shapefiles
directory = "results/NE/"

#suffix on the summer irriigated area shapefiles
suffix = "_Irrigated_LS_masked"

#name of area of interest
AOI = "Tasmania_Summer"

#results path for exported csv
results_csv = "results/plots/NE_annual_area.csv"

In [ ]:
#list of years to help for-loop iterate through folders
x = range(2000,2019,1)
years = []
for i in x:
    nextyear = str(i + 1)[2:]
    y = str(i) + "_" + nextyear
    years.append(str(y))
# removing years that didn't work
years =  [e for e in years if e not in ('2011_12', '2012_13')]
years.sort()

#setting up filepaths
folders = os.listdir(directory)
folders.sort()

#little function to calculate area
def getIrrigatedArea(shp):
    irr = gpd.read_file(shp)
    area_a.append(irr.area.sum() / 10000)

#grab sum of irrigated area for each each and add to list
area_a = []
for year, folder in zip(years, folders): 
    #progress indicator
    print("\r", "working on year: " + year, end = '')
    getIrrigatedArea(directory+folder+"/"+AOI+ year + suffix+".shp")

#convert years back into integers for plot
years_dt = []
for i in years:
    x = int(i[:-3])
    years_dt.append(x)
    
#create results to pandas dataframe and export csv
df = pd.DataFrame.from_dict({'year':years_dt, 'irrigated area':area_a})#, '0.75+0.8':area_b, '0.70+0.75+0.8':area_c})
df = df.set_index('year')
df.to_csv(results_csv, header=True)

In [ ]:
#plot annual irrigated area
fontsize = 17
title = "Tasmania NE Annual Summer Irrigated Area"
figsave = "results/plots/tasNE_annual_irrigated_area.pdf"

df = pd.read_csv(results_csv, index_col=0)
ax = df.plot(colormap='jet', linestyle='--', marker='o', legend=False,figsize=(20,8))
ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
ax.set_xlabel('Year (Summer Nov-Mar)', fontsize=fontsize)
ax.set_ylabel('Area under Irrigation ($10^4$ Ha)', fontsize=fontsize)
ax.set_ylim(bottom=-int((df['irrigated area'].max()*0.05)), top=(df['irrigated area'].max()+(df['irrigated area'].max()*0.1)))
ax.ticklabel_format(axis='y', style='sci', scilimits=(0, 3), useMathText=True)
plt.tick_params(labelsize=fontsize)
ax.grid(True, linestyle='--', alpha=0.75, color="gray")
ax.axvspan(2010, 2013, alpha=0.4, color='grey')
ax.set_facecolor('cornsilk')
ax.set_title(title, fontsize=fontsize)
#     plt.tight_layout()
plt.savefig(figsave,
           orientation='landscape')

### Annual new irrigation regions

In [ ]:
#-----------
#user inputs
#-----------

#where are the irrigated area shapefiles
directory = "results/NE/"

#name of area of interest
AOI = "Tasmania_Summer"

#path to a folder to store interim results
interim_results = "results/interim_NE/"

#path to a folder to store difference shapefiles
diff_results = "results/difference_NE/"

#path and name of csv file for results
newArea_csv = "results/plots/tasNE_new_area_added_each_year.csv"

#suffix on the summer irriigated area shapefiles
suffix = "_Irrigated_LS_masked"

#catchment name
catchment = 'NE'

#first year in the IA timeseries
firstyear = '2000'

In [ ]:
# create list of time strings for function
timerangelist = [str(t) for t in range(int(firstyear),2019,1)]
timerangelist = [e for e in timerangelist if e not in ('2011', '2012')]

In [ ]:
#Function to cumulatively union the IA shapefiles
def cumulativelyUnion(timerangelist, shapes):
    for t in timerangelist:
        print("\r", "working on "+t, end = '')
        #open the year and add it to a list
        nextyear= str(int(t)+1)[2:]
        irr = gpd.read_file(directory+AOI+t+"_"+nextyear+"/"+AOI+t+"_"+nextyear+suffix+".shp")
        shapes.append(irr)
        
        print("..length of [shapes] = "+str(len(shapes)))
        
        #now do the concatentation & dissolve
        x = pd.concat(shapes)
        x['DISS_ID'] = 1
        x = x.dissolve(by='DISS_ID', aggfunc='sum')
        x['year_range'] = firstyear+ "_" + t
        x['catchment'] = catchment
        #export to interim folder
        x.to_file(interim_results+AOI+"_"+catchment+"_"+firstyear+"_"+t+".shp")

#run the function
shapes=[]
cumulativelyUnion(timerangelist,shapes)

In [ ]:
# series of functions to find the 'new' area each year
def areaofFirstYear(area_list):
    #open the first timerange
    firstyeararea = gpd.read_file(interim_results+AOI+"_"+catchment+"_"+firstyear+"_"+firstyear+".shp")
    area_list.append((firstyeararea.area.sum()/ 10000))
        
def getdifferenceAreas(yr1, yr2, area_list):
    df1 = gpd.read_file(interim_results+AOI+"_"+catchment+"_"+firstyear+"_"+ yr1 + ".shp")
    df2 = gpd.read_file(interim_results+AOI+"_"+catchment+"_"+firstyear+"_"+ yr2 + ".shp")
    diff = gpd.overlay(df2, df1, how='difference')
    diff = diff.explode() #need to explode to filter for small areas
    diff['area_diff'] = diff['geometry'].area
    toosmall = diff['area_diff'] >= 100000 #less than 10ha
    diff = diff[toosmall]
    print("...exporting "+yr2+"_minus_"+yr1)
    diff.to_file(diff_results+ AOI+"_"+catchment+"_"+yr2+"_minus_"+yr1+".shp")
    area_list.append(float(diff.area.sum() / 10000))
        
def runpart2(area_list):
    areaofFirstYear(area_list)
    for i in range(1, len(timerangelist)):
        print("\r", "working on "+ timerangelist[i], end = '')
        getdifferenceAreas(timerangelist[i-1],timerangelist[i], area_list)

#run the functions
area_list = []
runpart2(area_list)

In [ ]:
# grab the 'difference' files, calculate area and export results as csv
diff = []
for file in os.listdir(diff_results):
    if file.endswith(".shp"):
        diff.append(os.path.join(diff_results, file))
diff.sort()

area_list = []
for i in diff:
    x = gpd.read_file(i)
    area_list.append((x.area.sum()/ 10000))
    
df = pd.DataFrame(index=timerangelist[1:])
df[AOI+"_"+catchment] = area_list
df.to_csv(newArea_csv, header=True)

### plotting new area added each year

First 3 years are removed to account for paddock rotations

In [ ]:
fontsize = 17
title = "Tasmania NE New Irrigated Areas"
figsave = "results/plots/tasNE_newAreaAdded.pdf"


df = pd.read_csv(newArea_csv, index_col=0)
df=df[2:]
ax = df.plot(colormap='jet', linestyle='--', marker='o', legend=False,figsize=(20,8))
# ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
ax.set_xlabel('Year Range', fontsize=fontsize)
ax.set_ylabel('Irrigatable Area ($10^3$ Ha)', fontsize=fontsize)
ax.ticklabel_format(axis='y', style='sci', scilimits=(0, 3), useMathText=True)
# ax.set_ylim(bottom=int((df['NMDB'].max()*0.35)), top=(df['NMDB'].max()+(df['NMDB'].max()*0.05)))
plt.tick_params(labelsize=fontsize)
ax.grid(True, linestyle='--', alpha=0.75, color="gray")
ax.axvspan(2010, 2013, alpha=0.4, color='grey')
ax.set_facecolor('cornsilk')
ax.set_title(title, fontsize=fontsize)
# plt.tight_layout()
plt.savefig(figsave,
           orientation='landscape')